# Full algorithm - generating grocery lists

1. Define variables: days-times-stores
2. Define limits: min amount-max amount
3. Generate (random) lists from narrowed down df

### Import libraries 

In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dataframe_image as dfi
from datetime import time
import matplotlib.dates as mdates
from matplotlib.ticker import StrMethodFormatter
from matplotlib.pyplot import figure
from random import sample

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

class bcolors:
    WARNING = '\033[91m'
    BOLD = '\033[1m'

In [7]:
    # Function to generate #visits per day of the week
from AlgorithmDAYS_HH2 import beforeNY, AfterNY
    # Function to generate store name (&type) per weekday (+ assign #of visits per day)
from AlgorithmSTORE_HH2 import Mondays, Tuesdays, Wednesdays, Thursdays, Fridays, Saturdays, Sundays
    #Function to define the limits for total visits, days, same store and type per week
from AlgorithmCOUNTS_HH2 import CountTotalVisits, CountTotalDays, CountStoreName, CountStoreType, CountTotalPerday, CountVisitsPerDay
    # Function to decide time (morning, noon. afternoon) per visited store
from AlgorithmTIMES_HH2 import AlbertHeijn, Okay, Delhaize, Carrefour, VersavelPoelman, Sys, Brabo, Ikea, Kruidvat
    # Function to decide amounts per visit & per week
from AlgorithmAMOUNT_HH2 import StoreAmount, WeekAmount, StoreCount, WeekCount

The P-Value of the ChiSq Test is: 0.05802400602619058
The P-Value of the ChiSq Test is: 0.05787497524814351
The P-Value of the ChiSq Test is: 0.20057647290101452
The P-Value of the ChiSq Test is: 0.018746113254959878
The P-Value of the ChiSq Test 1 is: 0.2886349897006456
The P-Value of the ChiSq Test 2 is: 0.3168531788968301


In [8]:
dataframe = pd.read_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2.csv")

# Define variables - day/time/store

## Days & visits

In [9]:
CountTotalVisits()

(4, 8)

In [10]:
CountTotalDays()

(2, 5)

In [13]:
i = 0

while i < 2:

    ### 1. What DAY?

        # Select the relevant df for this generated grocery list, comment out the other one
    # df = beforeNY()
    df = AfterNY()

        # identify counts for visits per weekday
    mo = int(df["times"].values[0])
    tu = int(df["times"].values[1])
    we = int (df["times"].values[2])
    th = int (df["times"].values[3])
    fr = int (df["times"].values[4])
    sa = int (df["times"].values[5])
    su = int (df["times"].values[6])
    
        
        ### CHECKPOINT: range for visits/week and days/week
        
        # Check if the total GROCERY VISITS PER WEEK are within the normal range
    visitcount = df['times'].sum()
    totalvisits = CountTotalVisits()
    minvisits = totalvisits[0]
    maxvisits = totalvisits[1]
    if minvisits <= visitcount <= maxvisits:
        i= i+1
    else:
        print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few/many visits per week")
        i=0
        continue

        # Check if the total GROCERY DAYS PER WEEK are within the normal range
    dayscount = df['times'].nunique()
    totaldays = CountTotalDays()
    mindays = totaldays[0]
    maxdays = totaldays[1]
    if mindays <= dayscount <= maxdays:
        i = i+1
        display(df)
    else:
        print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few/many days per week")
        i=0
        continue

,times,day
4,2.0,Monday
6,0.0,Tuesday
4,0.0,Wednesday
4,0.0,Thursday
7,1.0,Friday
6,1.0,Saturday
4,0.0,Sunday


## Stores & time

In [16]:
restart = True 

while restart:

    ### 2. What STORE?

        # Generate store names for the [previously identified] shopping dags + corresponding store type
    stores = [Mondays(mo), Tuesdays(tu), Wednesdays(we), Thursdays(th), Fridays(fr), Saturdays(sa), Sundays(su)]
        # create one df for all visited stores + day of the week
    stores = pd.concat(stores)


    ### 3. What TIME?

        # Identifier counts for store names per week
    AHt = (stores['store_name'] == 'Albert Heijn').sum()
    OKt = (stores['store_name'] == 'Okay').sum()
    DHt = (stores['store_name'] == 'Delhaize').sum()
    CFt = (stores['store_name'] == 'Carrefour').sum()
    VPt = (stores['store_name'] == 'Versavel Poelman').sum()
    SYt = (stores['store_name'] == 'Sys').sum()
    BRt = (stores['store_name'] == 'Brabo').sum()
    IKt = (stores['store_name'] == 'Ikea').sum()
    KRt = (stores['store_name'] == 'Kruidvat').sum()

        # Generate times per storename and compile in df with stores
    storetimes = [AlbertHeijn(AHt), Okay(OKt), Delhaize(DHt), Carrefour(CFt), VersavelPoelman(VPt), Sys(SYt), Brabo(BRt), Ikea(IKt), Kruidvat(KRt)]
    storetimes = pd.concat(storetimes)
    stores = pd.merge(stores, storetimes, on="store_name")

    ### CHECKPOINT: range for amount of stores per week.day

        # Check if the total STORE TYPES VISITED PER WEEK are within the normal range
    for storetype in dataframe['store_type']:
        typecount = stores[stores['store_type']==storetype]['store_type'].count()
        countstoretype = CountStoreType(storetype)
        minstoretype = countstoretype[0]
        maxstoretype = countstoretype[1]
        if typecount < minstoretype:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few store type:", storetype)
            restart = True
            break # force restart
        elif maxstoretype < typecount:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too many store type:", storetype)
            restart = True
            break # force restart
        else:
            pass


        # Check if the total STORE NAMES VISITED PER WEEK are within the normal range
    for storename in dataframe['store_name']:
        namecount = stores[stores['store_name']==storename]['store_name'].count()
        countstorename = CountStoreName(storename)
        minstorename = countstorename[0]
        maxstorename = countstorename[1]
        if namecount < minstorename:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few store name:", storename)
            restart = True
            break # force restart
        elif maxstorename < namecount:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too many store name:", storename)
            restart = True
            break # force restart
        else:
            pass


        # Check if the total STORES VISITED PER DAY are within the normal range
    for day in dataframe['day']:
        perdaycount = stores[stores['day']==day]['day'].count()
        CountTotalPerday = CountVisitsPerDay(day)
        minstoreperday = CountTotalPerday[0]
        maxstoreperday = CountTotalPerday[1]
        if namecount < minstoreperday:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few stores on:", day)
            restart = True
            break # force restart
        elif maxstoreperday < namecount:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too many stores on:", day)
            restart = True
            break # force restart
        else:
            restart = False
            display(stores)
            break

 RERUN - Too many stores on: Saturday
 RERUN - Too many stores on: Saturday
 RERUN - Too many stores on: Saturday


,store_name,store_type,day,time
0,Versavel Poelman,butcher,Monday,morning
1,Albert Heijn,supermarket,Monday,noon
2,Delhaize,supermarket,Friday,morning
3,Okay,supermarket,Saturday,afternoon


## Define limits per week (amount)

In [17]:
storetime = stores[['store_name','time']]
storetime

restart = True 

while restart:
    storecounts = []

    for store, time in storetime.itertuples(index=False):
        storecounts.append(StoreCount(store,time))

    stores['counts'] = storecounts
    
    # Check if the total STORES VISITED PER DAY are within the normal range
    for items in dataframe['item_type']:
        week9count = stores['counts'].sum()
        NormWeekCount = WeekCount()
        minWeekCount = NormWeekCount[0]
        maxWeekCount = NormWeekCount[1]
        if week9count < minWeekCount:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few items:", week9count)
            restart = True
            break # force restart
        elif maxWeekCount < week9count:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too many items:", week9count)
            restart = True
            break # force restart
        else:
            restart = False
            display(stores)
            break

,store_name,store_type,day,time,counts
0,Versavel Poelman,butcher,Monday,morning,7
1,Albert Heijn,supermarket,Monday,noon,2
2,Delhaize,supermarket,Friday,morning,34
3,Okay,supermarket,Saturday,afternoon,5


# The Grocery Lists

In [18]:
df = pd.read_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2.csv")

In [41]:
    ## Create dataframes for each day they go shopping
daydf = {}

for day in stores['day']:
        storeday = df[df['day'] == day]
        items = storeday[['item_type', 'day', 'store_name', 'amount']]
        daydf[day] = pd.DataFrame(items)
        daydf[day].drop_duplicates()


    ## Split up the dataframes to store name for mon, tue, sat, sun
storenames = stores.store_name.values.tolist()
days = stores.day.values.tolist()
                
storenamedf = {}

if 'Monday' or 'Tuesday' or 'Satuday' or 'Sunday' in stores.values:
        for store, day  in zip(storenames, days):
            thisdaydf = daydf[day]
            storesdf = thisdaydf[thisdaydf['store_name'] == store]
            items = storesdf[['item_type', 'day', 'store_name', 'amount']]
            storenamedf[store] = pd.DataFrame(items)
            storenamedf[store].drop_duplicates()

In [43]:
storenamedf['Versavel Poelman']

,item_type,day,store_name,amount
203,meat spread,Monday,Versavel Poelman,1
204,charcuterie,Monday,Versavel Poelman,1
205,cheese,Monday,Versavel Poelman,1


In [42]:
    ## Sample the dataframes per day they go shopping
dayamount = stores[['day', 'store_name', 'counts']]
dayamount

if 'Monday' in dayamount.values:
        grocerylist = dayamount[dayamount['day'] == 'Monday']
        storenameslist = grocerylist.store_name.values.tolist()
        amounts = grocerylist.counts.values.tolist()
    for amount, storename in zip(amounts, storenameslist):
        if 'Monday' in dayamount.values:
            storenamedf[storename]= storenamedf[storename].drop_duplicates()
            display(storenamedf[storename].sample(n=amount, replace=True))
        else:
            pass
        
if 'Tuesday' in dayamount.values:
        grocerylist = dayamount[dayamount['day'] == 'Tuesday']
        storenameslist = grocerylist.store_name.values.tolist()
        amounts = grocerylist.counts.values.tolist()
    for amount, storename in zip(amounts, storenameslist):
        if 'Tuesday' in dayamount.values:
            storenamedf[storename]= storenamedf[storename].drop_duplicates()
            display(storenamedf[storename].sample(n=amount, replace=True))
        else:
            pass

if 'Wednesday' in dayamount.values:
        grocerylist = dayamount[dayamount['day'] == 'Wednesday']
    for amount in grocerylist['counts']:
        if 'Wednesday' in dayamount.values:
            daydf['Wednesday'] = daydf['Wednesday'].drop_duplicates()
            display(daydf['Wednesday'].sample(n=amount, replace=False))
        else:
            pass

if 'Thursday' in dayamount.values:
        grocerylist = dayamount[dayamount['day'] == 'Thursday']
    for amount in grocerylist['counts']:
        if 'Thursday' in dayamount.values:
            daydf['Thursday'] = daydf['Thursday'].drop_duplicates()
            display(daydf['Thursday'].sample(n=amount, replace=False))
        else:
            pass    
        
if 'Friday' in dayamount.values:
        grocerylist = dayamount[dayamount['day'] == 'Friday']
    for amount in grocerylist['counts']:
        if 'Friday' in dayamount.values:
            daydf['Friday'] = daydf['Friday'].drop_duplicates()
            display(daydf['Friday'].sample(n=amount, replace=False))
        else:
            pass  
        
if 'Saturday' in dayamount.values:
        grocerylist = dayamount[dayamount['day'] == 'Saturday']
        storenameslist = grocerylist.store_name.values.tolist()
        amounts = grocerylist.counts.values.tolist()
    for amount, storename in zip(amounts, storenameslist):
        if 'Saturday' in dayamount.values:
            storenamedf[storename]= storenamedf[storename].drop_duplicates()
            display(storenamedf[storename].sample(n=amount, replace=True))
        else:
            pass
        
if 'Sunday' in dayamount.values:
        grocerylist = dayamount[dayamount['day'] == 'Sunday']
        storenameslist = grocerylist.store_name.values.tolist()
        amounts = grocerylist.counts.values.tolist()
    for amount, storename in zip(amounts, storenameslist):
        if 'Sunday' in dayamount.values:
            storenamedf[storename]= storenamedf[storename].drop_duplicates()
            display(storenamedf[storename].sample(n=amount, replace=False))
        else:
            pass

,item_type,day,store_name,amount
205,cheese,Monday,Versavel Poelman,1
204,charcuterie,Monday,Versavel Poelman,1
205,cheese,Monday,Versavel Poelman,1
205,cheese,Monday,Versavel Poelman,1
203,meat spread,Monday,Versavel Poelman,1
205,cheese,Monday,Versavel Poelman,1
204,charcuterie,Monday,Versavel Poelman,1


,item_type,day,store_name,amount
342,vegetable box,Monday,Albert Heijn,1
358,chicken,Monday,Albert Heijn,1


,item_type,day,store_name,amount
80,cucumber,Friday,Delhaize,1
371,lunch salad,Friday,Albert Heijn,1
60,chocolate milk,Friday,Delhaize,1
84,fish spread,Friday,Delhaize,1
61,pork,Friday,Delhaize,1
164,tea,Friday,Ikea,1
85,charcuterie,Friday,Delhaize,1
65,alcoholic drinks,Friday,Delhaize,1
92,oranges,Friday,Delhaize,1
74,broccoli,Friday,Delhaize,1


,item_type,day,store_name,amount
258,grapes,Saturday,Okay,1
268,apple sauce,Saturday,Okay,1
257,chicken,Saturday,Okay,1
250,eggs,Saturday,Okay,1
260,spring roll,Saturday,Okay,1
